In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install transformers[torch]

In [ ]:
from datasets import load_dataset, DatasetDict
dataset  = load_dataset('HuggingFaceH4/ultrachat_200k',split=['train_sft', 'test_sft'])
dataset = DatasetDict(
    {
        "train": dataset[0],
        "test": dataset[1],
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 207865
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 23110
    })
})

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, AutoModelForCausalLM
pretrained_model = "facebook/opt-125m"
model = AutoModelForCausalLM.from_pretrained(pretrained_model)

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
message_contents = []
for row in dataset["train"]:
    # Extracting message contents from list of dictionaries and joining them into a single string
    message_content = " ".join([msg["content"] for msg in row["messages"]])
    message_contents.append(message_content)

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

max_length = 128

def tokenize_function(examples):
    return tokenizer(message_contents[:5],max_length=max_length)


# Apply the tokenization function to the entire dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=10,
    remove_columns=dataset["train"].column_names
)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Map:   0%|          | 0/207865 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/23110 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 103935
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 11555
    })
})

In [ ]:
tokenized_dataset['train']['input_ids'][0]


[2,
 4528,
 9223,
 3253,
 7,
 2810,
 12,
 805,
 12049,
 36,
 44036,
 1790,
 2088,
 231,
 4,
 288,
 30787,
 9944,
 1243,
 204,
 4,
 288,
 30787,
 2884,
 1250,
 3631,
 155,
 4,
 288,
 2744,
 30026,
 132,
 4,
 288,
 30787,
 20648,
 10990,
 195,
 4,
 288,
 2744,
 322,
 653,
 4782,
 1732,
 524,
 38,
 634,
 116,
 50118,
 4148,
 110,
 42121,
 6052,
 359,
 17619,
 42121,
 9042,
 6,
 47,
 64,
 2773,
 311,
 5,
 5929,
 2274,
 9,
 10,
 1152,
 15,
 30731,
 30,
 10298,
 65,
 9,
 5,
 4782,
 18,
 1490,
 12,
 179,
 9629,
 328,
 50118,
 12861,
 15937,
 6052,
 359,
 17619,
 42121,
 9042,
 40,
 122,
 2332,
 5,
 5929,
 1152,
 2274,
 95,
 30,
 24752,
 81,
 14,
 1152,
 2274,
 45665,
 4,
 50118,
 27847,
 42,
 1905,
 3253,
 7,
 70,
 9042,
 9,
 5,
 4782,
 50,
 95,
 2167,
 1980,
 25,
 3147,
 11,
 5,
 2788,
 1468]

In [ ]:
decoded_string = tokenizer.decode([2,
 4528,
 9223,
 3253,
 7,
 2810,
 12,
 805,
 12049,
 36,
 44036,
 1790,
 2088,
 231,
 4,
 288,
 30787,
 9944,
 1243,
 204,
 4,
 288,
 30787,
 2884,
 1250,
 3631,
 155,
 4,
 288,
 2744,
 30026,
 132,
 4,
 288,
 30787,
 20648,
 10990,
 195,
 4,
 288,
 2744,
 322,
 653,
 4782,
 1732,
 524,
 38,
 634,
 116,
 50118,
 4148,
 110,
 42121,
 6052,
 359,
 17619,
 42121,
 9042,
 6,
 47,
 64,
 2773,
 311,
 5,
 5929,
 2274,
 9,
 10,
 1152,
 15,
 30731,
 30,
 10298,
 65,
 9,
 5,
 4782,
 18,
 1490,
 12,
 179,
 9629,
 328,
 50118,
 12861,
 15937,
 6052,
 359,
 17619,
 42121,
 9042,
 40,
 122,
 2332,
 5,
 5929,
 1152,
 2274,
 95,
 30,
 24752,
 81,
 14,
 1152,
 2274,
 45665,
 4,
 50118,
 27847,
 42,
 1905,
 3253,
 7,
 70,
 9042,
 9,
 5,
 4782,
 50,
 95,
 2167,
 1980,
 25,
 3147,
 11,
 5,
 2788,
 1468])
print(decoded_string)

</s>These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    "bertQA",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    # fp16=is_gpu_available,
    push_to_hub=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # hub_model_id=huggingface_reponame
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.012800,0.012702


TrainOutput(global_step=6496, training_loss=0.01902560236418776, metrics={'train_runtime': 3452.1758, 'train_samples_per_second': 30.107, 'train_steps_per_second': 1.882, 'total_flos': 6789346836480000.0, 'train_loss': 0.01902560236418776, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.012701951898634434,
 'eval_runtime': 115.1488,
 'eval_samples_per_second': 100.348,
 'eval_steps_per_second': 6.279,
 'epoch': 1.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1717699763.a461284479d3.831.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/krushnakant27/bertQA/commit/7bc54d3513a4e09cd713d37c194da4c022dccb07', commit_message='End of training', commit_description='', oid='7bc54d3513a4e09cd713d37c194da4c022dccb07', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# from transformers import GPT2Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
# max_length = 128

# # Function to extract and tokenize the 'content' from the 'messages' column
# def tokenize_function(examples):
#     # Extract the 'content' field from the 'messages' column
#     contents = [json.loads(message)['content'] if isinstance(message, str) else message['content'] for message in examples['messages']]
#     return tokenizer(contents, max_length=max_length, truncation=True, padding="max_length")

# # Apply the tokenization function to the entire dataset
# tokenized_dataset = dataset.map(
#     tokenize_function,
#     batched=True,
#     batch_size=10,
#     remove_columns=dataset["train"].column_names
# )

# # Verify the tokenized content
# print(tokenized_dataset['train'].features)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_name = "krushnakant27/bertQA"  # Replace with the model's name or path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Prepare input data
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text
outputs = model.generate(
    **inputs,
    max_length=50,  # Maximum length of the generated text
    num_return_sequences=1,  # Number of generated sequences to return
    do_sample=True,  # Use sampling; otherwise, greedy decoding is used
    top_k=50,  # Top-K sampling
    top_p=0.95,  # Top-P (nucleus) sampling
    temperature=0.7,  # Sampling temperature
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Once upon a time, I was a student of the University of California, and had been planning a visit to the campus to study the history of the university. While there are undoubtedly millions of words of our own to describe the university's history,
